In [41]:
import pandas as pd
import nba_api
import altair as alt
import streamlit as st
from itertools import product
import nba_api.stats.endpoints
from datetime import datetime
import time
from requests.exceptions import ReadTimeout, RequestException

In [55]:
league_schedule = nba_api.stats.endpoints.ScheduleLeagueV2(
    league_id = '00',
    season = '2025-26'
).get_data_frames()[0]

today = datetime.now().strftime('%Y-%m-%d')

league_schedule['dateGame'] = pd.to_datetime(league_schedule['gameDate'])
league_schedule['matchup'] = league_schedule['awayTeam_teamTricode'] + ' @ ' + league_schedule['homeTeam_teamTricode']

todays_games = league_schedule[
    league_schedule['dateGame'] == today
][[
    'dateGame', 'gameId', 'gameCode', 'matchup', 'homeTeam_teamTricode', 'awayTeam_teamTricode', 'homeTeam_score', 'awayTeam_score',
    'homeTeam_wins', 'homeTeam_losses', 'homeTeam_seed', 'awayTeam_wins', 'awayTeam_losses', 'awayTeam_seed'
]]

todays_matchups = todays_games['matchup'].unique().tolist()

todays_games

,dateGame,gameId,gameCode,matchup,homeTeam_teamTricode,awayTeam_teamTricode,homeTeam_score,awayTeam_score,homeTeam_wins,homeTeam_losses,homeTeam_seed,awayTeam_wins,awayTeam_losses,awayTeam_seed
143,2025-10-31,0022500020,20251031/ATLIND,ATL @ IND,IND,ATL,0,0,0,4,0,2,3,0
144,2025-10-31,0022500021,20251031/BOSPHI,BOS @ PHI,PHI,BOS,0,0,4,0,0,2,3,0
145,2025-10-31,0022500022,20251031/TORCLE,TOR @ CLE,CLE,TOR,0,0,3,2,0,1,4,0
146,2025-10-31,0022500023,20251031/NYKCHI,NYK @ CHI,CHI,NYK,0,0,4,0,0,2,2,0
147,2025-10-31,0022500024,20251031/LALMEM,LAL @ MEM,MEM,LAL,0,0,3,2,0,3,2,0
148,2025-10-31,0022500025,20251031/UTAPHX,UTA @ PHX,PHX,UTA,0,0,1,4,0,2,2,0
149,2025-10-31,0022500026,20251031/DENPOR,DEN @ POR,POR,DEN,0,0,3,2,0,3,1,0
150,2025-10-31,0022500027,20251031/NOPLAC,NOP @ LAC,LAC,NOP,0,0,2,2,0,0,4,0


,dateGame,gameId,gameCode,matchup,homeTeam_teamTricode,awayTeam_teamTricode,homeTeam_score,awayTeam_score,homeTeam_wins,homeTeam_losses,homeTeam_seed,awayTeam_wins,awayTeam_losses,awayTeam_seed,pointsLeaders_0_firstName,pointsLeaders_0_lastName
143,2025-10-31,0022500020,20251031/ATLIND,ATL @ IND,IND,ATL,0,0,0,4,0,2,3,0,None,None
144,2025-10-31,0022500021,20251031/BOSPHI,BOS @ PHI,PHI,BOS,0,0,4,0,0,2,3,0,None,None
145,2025-10-31,0022500022,20251031/TORCLE,TOR @ CLE,CLE,TOR,0,0,3,2,0,1,4,0,None,None
146,2025-10-31,0022500023,20251031/NYKCHI,NYK @ CHI,CHI,NYK,0,0,4,0,0,2,2,0,None,None
147,2025-10-31,0022500024,20251031/LALMEM,LAL @ MEM,MEM,LAL,0,0,3,2,0,3,2,0,None,None
148,2025-10-31,0022500025,20251031/UTAPHX,UTA @ PHX,PHX,UTA,0,0,1,4,0,2,2,0,None,None
149,2025-10-31,0022500026,20251031/DENPOR,DEN @ POR,POR,DEN,0,0,3,2,0,3,1,0,None,None
150,2025-10-31,0022500027,20251031/NOPLAC,NOP @ LAC,LAC,NOP,0,0,2,2,0,0,4,0,None,None


In [28]:
# Create a dataframe with all column names from league_schedule
column_names_df = pd.DataFrame(league_schedule.columns, columns=['Column Name'])

# Set pandas display options to show all rows
pd.set_option('display.max_rows', None)

# Display the dataframe
column_names_df.to_csv("~/Downloads/columns.csv")


In [14]:
current_season = '2025-26'
standings = nba_api.stats.endpoints.LeagueStandings(league_id='00', season=current_season, season_type='Regular Season').get_data_frames()[0]

standings

,LeagueID,SeasonID,TeamID,TeamCity,TeamName,Conference,ConferenceRecord,PlayoffRank,ClinchIndicator,Division,...,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,PreAS,PostAS
0,00,22025,1610612760,Oklahoma City,Thunder,West,3-0,1,,Northwest,...,None,None,None,None,None,6-0,None,None,6-0,None
1,00,22025,1610612741,Chicago,Bulls,East,3-0,1,,Central,...,None,None,None,None,None,4-0,None,None,4-0,None
2,00,22025,1610612755,Philadelphia,76ers,East,4-0,2,,Atlantic,...,None,None,None,None,None,4-0,None,None,4-0,None
3,00,22025,1610612759,San Antonio,Spurs,West,2-0,2,,Southwest,...,None,None,None,None,None,5-0,None,None,5-0,None
4,00,22025,1610612743,Denver,Nuggets,West,3-1,3,,Northwest,...,None,None,None,None,None,3-1,None,None,3-1,None
5,00,22025,1610612749,Milwaukee,Bucks,East,3-1,3,,Central,...,None,None,None,None,None,4-1,None,None,4-1,None
6,00,22025,1610612748,Miami,Heat,East,2-1,4,,Southeast,...,None,None,None,None,None,3-2,None,None,3-2,None
7,00,22025,1610612744,Golden State,Warriors,West,4-1,4,,Pacific,...,None,None,None,None,None,4-2,None,None,4-2,None
8,00,22025,1610612757,Portland,Trail Blazers,West,3-2,5,,Northwest,...,None,None,None,None,None,3-2,None,None,3-2,None
9,00,22025,1610612739,Cleveland,Cavaliers,East,3-2,5,,Central,...,None,None,None,None,None,3-2,None,None,3-2,None


In [15]:
standings.columns

Index(['LeagueID', 'SeasonID', 'TeamID', 'TeamCity', 'TeamName', 'Conference',
       'ConferenceRecord', 'PlayoffRank', 'ClinchIndicator', 'Division',
       'DivisionRecord', 'DivisionRank', 'WINS', 'LOSSES', 'WinPCT',
       'LeagueRank', 'Record', 'HOME', 'ROAD', 'L10', 'Last10Home',
       'Last10Road', 'OT', 'ThreePTSOrLess', 'TenPTSOrMore', 'LongHomeStreak',
       'strLongHomeStreak', 'LongRoadStreak', 'strLongRoadStreak',
       'LongWinStreak', 'LongLossStreak', 'CurrentHomeStreak',
       'strCurrentHomeStreak', 'CurrentRoadStreak', 'strCurrentRoadStreak',
       'CurrentStreak', 'strCurrentStreak', 'ConferenceGamesBack',
       'DivisionGamesBack', 'ClinchedConferenceTitle', 'ClinchedDivisionTitle',
       'ClinchedPlayoffBirth', 'EliminatedConference', 'EliminatedDivision',
       'AheadAtHalf', 'BehindAtHalf', 'TiedAtHalf', 'AheadAtThird',
       'BehindAtThird', 'TiedAtThird', 'Score100PTS', 'OppScore100PTS',
       'OppOver500', 'LeadInFGPCT', 'LeadInReb', 'FewerTurnove

In [11]:
# Create the synergy dataframe
synergy_playtypes =['Cut', 'Handoff', 'Isolation', 'Misc', 'OffScreen', 'Postup', 'PRBallHandler', 'PRRollman', 'OffRebound', 'Spotup', 'Transition']
synergy_sides = ['Offense', 'Defense']

def fetch_synergy_data(playtype, side_ball, max_retries=3, timeout=60):
    """
    Fetch synergy data with retry logic and timeout handling
    
    Args:
        playtype: The play type to fetch
        side_ball: 'Offense' or 'Defense'
        max_retries: Number of retry attempts
        timeout: Request timeout in seconds
    """
    if side_ball == 'Offense':
        side_ball = 'offensive'
    else:
        side_ball = 'defensive'

    for attempt in range(max_retries):
        try:
            synergy_data = nba_api.stats.endpoints.SynergyPlayTypes(
                league_id = '00',
                per_mode_simple = 'Totals',
                season = '2025-26',
                season_type_all_star = 'Regular Season',
                player_or_team_abbreviation = 'T',
                type_grouping_nullable = side_ball,
                play_type_nullable = playtype,
                timeout = timeout
            ).get_data_frames()[0]
            
            return synergy_data
            
        except (ReadTimeout, RequestException) as e:
            if attempt < max_retries - 1:
                wait_time = (attempt + 1) * 5  # Exponential backoff: 5s, 10s, 15s
                print(f"Timeout/Error fetching {playtype} - {side_ball}. Retrying in {wait_time} seconds... (Attempt {attempt + 1}/{max_retries})")
                time.sleep(wait_time)
            else:
                print(f"Failed to fetch {playtype} - {side_ball} after {max_retries} attempts. Error: {str(e)}")
                raise
        except Exception as e:
            print(f"Unexpected error fetching {playtype} - {side_ball}: {str(e)}")
            raise
    
    return None

In [12]:
# All combinations: 11 playtypes × 2 sides = 22 rows
# Add delays between requests to avoid rate limiting
synergy_dataframes = []

total_requests = len(synergy_playtypes) * len(synergy_sides)
current_request = 0

for playtype, side in product(synergy_playtypes, synergy_sides):
    current_request += 1
    print(f"Fetching {playtype} - {side} ({current_request}/{total_requests})...")
    
    try:
        df = fetch_synergy_data(playtype, side)
        synergy_dataframes.append(df)
        print(f"✓ Successfully fetched {playtype} - {side}")
        
        # Add delay between requests to avoid rate limiting (1-2 seconds)
        if current_request < total_requests:  # Don't sleep after the last request
            time.sleep(1.5)
            
    except Exception as e:
        print(f"✗ Error fetching {playtype} - {side}: {str(e)}")
        # Continue with other requests even if one fails
        continue

if synergy_dataframes:
    synergy_data = pd.concat(synergy_dataframes, ignore_index=True)
    print(f"\n✓ Successfully fetched {len(synergy_dataframes)}/{total_requests} datasets")
    synergy_data
else:
    print("✗ No data was successfully fetched. Please check your connection and try again.")

Fetching Cut - Offense (1/22)...
Timeout/Error fetching Cut - offensive. Retrying in 5 seconds... (Attempt 1/3)
Timeout/Error fetching Cut - offensive. Retrying in 10 seconds... (Attempt 2/3)
Failed to fetch Cut - offensive after 3 attempts. Error: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=60)
✗ Error fetching Cut - Offense: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=60)
Fetching Cut - Defense (2/22)...
Timeout/Error fetching Cut - defensive. Retrying in 5 seconds... (Attempt 1/3)
Timeout/Error fetching Cut - defensive. Retrying in 10 seconds... (Attempt 2/3)
Failed to fetch Cut - defensive after 3 attempts. Error: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=60)
✗ Error fetching Cut - Defense: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=60)
Fetching Handoff - Offense (3/22)...
Timeout/Error fetching Handoff - offensive. Retrying 

In [ ]:
synergy_data.to_csv('files/synergy_data_26.csv', index=False)

In [ ]:
# Add rank column for each play_type and type_grouping combination
# Rank by GP (Games Played) - you can change this to any column you want to rank by

synergy_offense = synergy_data[synergy_data['TYPE_GROUPING'] == 'Offensive']
synergy_offense['OFF_RANK'] = synergy_offense.groupby(['PLAY_TYPE', 'TYPE_GROUPING'])['PPP'].rank(ascending=False, method='dense')

synergy_defense = synergy_data[synergy_data['TYPE_GROUPING'] == 'Defensive']
synergy_defense['DEF_RANK'] = synergy_defense.groupby(['PLAY_TYPE', 'TYPE_GROUPING'])['PPP'].rank(ascending=True, method='dense')



# Display the data with ranks
synergy_offense[['PLAY_TYPE', 'TYPE_GROUPING', 'TEAM_NAME', 'PPP', 'OFF_RANK']].sort_values(['PLAY_TYPE', 'TYPE_GROUPING', 'OFF_RANK']).reset_index(drop=True)

/var/folders/yz/j96xl3wd5ll5m4p0qwtc4tz00000gn/T/ipykernel_74638/2364116158.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  synergy_offense['OFF_RANK'] = synergy_offense.groupby(['PLAY_TYPE', 'TYPE_GROUPING'])['PPP'].rank(ascending=False, method='dense')
/var/folders/yz/j96xl3wd5ll5m4p0qwtc4tz00000gn/T/ipykernel_74638/2364116158.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  synergy_defense['DEF_RANK'] = synergy_defense.groupby(['PLAY_TYPE', 'TYPE_GROUPING'])['PPP'].rank(ascending=True, method='den

,PLAY_TYPE,TYPE_GROUPING,TEAM_NAME,PPP,OFF_RANK
0,Cut,Offensive,Cleveland Cavaliers,1.427,1.0
1,Cut,Offensive,Chicago Bulls,1.389,2.0
2,Cut,Offensive,Denver Nuggets,1.380,3.0
3,Cut,Offensive,Boston Celtics,1.364,4.0
4,Cut,Offensive,LA Clippers,1.358,5.0
...,...,...,...,...,...
325,Transition,Offensive,Brooklyn Nets,1.081,23.0
326,Transition,Offensive,Washington Wizards,1.072,24.0
327,Transition,Offensive,Toronto Raptors,1.051,25.0
328,Transition,Offensive,Portland Trail Blazers,1.026,26.0
